In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Activation

from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import average_precision_score

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import cv2
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import glob, os, shutil

import datetime, time
import pickle

from IPython.display import display
from IPython.display import clear_output

In [ ]:
target_names = ['Not Whale', 'Whale']
learning_rates = [0.000001,0.0000001,0.00000001,0.000000001]

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,   # randomly rotate between 0-180 degrees,
    horizontal_flip=True, # allows to randomly flip the image Horizontally
    vertical_flip=True    # allows to randomly flip the image Vertically
)

## MobileNetV2

In [ ]:
train_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Train',
    batch_size = 2,
    target_size = (224, 224),
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Test',
    batch_size = 2,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle = False
)

for i, learning_rate in enumerate(learning_rates):
    clear_output()
    print(str(learning_rate))
    try:
        details = {
            "learning_rate": learning_rate
        }

        base_model = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = True

        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            Activation('relu'),
            layers.Dense(2, activation = 'softmax'),
        ])

        model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate),
                      metrics=['accuracy'])

        start = time.time()

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "model_checkpoints/mobile_net_V2_"+str(learning_rate)+"{epoch:02d}.h5",
            save_weights_only=True,
            save_freq='epoch'
        )

        history = model.fit(train_generator,
                            epochs = 20,
                            validation_data=validation_generator,
                            callbacks = [checkpoint],
                            verbose = 1)

        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time Taken: {:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds))

        model.save_weights(filepath = "trained_models/mobile_net_V2_"+str(learning_rate)+".h5")
        pickle.dump( history.history, open("trained_models/mobile_net_V2_history_"+str(learning_rate)+".p", "wb" ) )

        details["model_path"] = "trained_models/mobile_net_V2_"+str(learning_rate)+".h5"
        details["history_path"] = "trained_models/mobile_net_V2_history_"+str(learning_rate)+".p"
        details["time_taken"] = end-start
        details["time_taken_hrs"] = "{:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds)

        mobilenet_models.append(details)
    except:
        pass

In [ ]:
df = pd.DataFrame(mobilenet_models)
df.to_csv("mobilenet_results.csv", index = False)

## XceptionV1

In [ ]:
train_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Train',
    batch_size = 2,
    target_size = (224, 224),
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Test',
    batch_size = 2,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle = False
)

for i, learning_rate in enumerate([0.000001,0.00000001,0.000000001]):
    clear_output()
    print(learning_rate)
    try:
        details = {
            "learning_rate": learning_rate
        }

        base_model = tf.keras.applications.Xception(input_shape = (224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = True

        global_average_layer = layers.GlobalAveragePooling2D()

        classes_layer = layers.Dense(2, activation = 'sigmoid')

        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            classes_layer,
        ])

        model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      optimizer = tf.keras.optimizers.RMSprop(lr = learning_rate), #0.00000001
                      metrics=['accuracy'])

        start = time.time()

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "model_checkpoints/xception_"+str(learning_rate)+"{epoch:02d}.h5",
            save_weights_only=True,
            save_freq='epoch'
        )

        history = model.fit(train_generator,
                            epochs = 20,
                            validation_data=validation_generator,
                            callbacks = [checkpoint],
                            verbose = 1)

        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time Taken: {:0>2}:{:0>2}:{:05.3f}".format(int(hours), int(minutes), seconds))

        model.save_weights(filepath = "trained_models/xception_"+str(learning_rate)+".h5")
        pickle.dump(history.history, open("trained_models/xception_history_"+str(learning_rate)+".p", "wb" ))

        details["model_path"] = "trained_models/xception_"+str(learning_rate)+".h5"
        details["history_path"] = "trained_models/xception_history_"+str(learning_rate)+".p"
        details["time_taken"] = end-start
        details["time_taken_hrs"] = "{:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds)

        xception_models.append(details)
        
        df = pd.DataFrame(xception_models)
        df.to_csv("xception_results.csv", index = False)
        
    except:
        pass
    
clear_output()

## Resnet152-V2

In [ ]:
train_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Train',
    batch_size = 2,
    target_size = (224, 224),
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Test',
    batch_size = 2,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle = False
)

for i, learning_rate in enumerate([0.000001,0.0000001,0.00000001]):
    clear_output()
    print(learning_rate)
    try:
        details = {
            "learning_rate": learning_rate
        }

        base_model = tf.keras.applications.ResNet152V2(input_shape = (224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = True

        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            Activation('relu'),
            layers.Dense(2, activation = 'softmax'),
        ])

        model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate),
                      metrics=['accuracy'])

        start = time.time()

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "model_checkpoints/resnet152_V2_"+str(learning_rate)+"{epoch:02d}.h5",
            save_weights_only=True,
            save_freq='epoch'
        )

        history = model.fit(train_generator,
                            epochs = 20,
                            validation_data=validation_generator,
                            callbacks = [checkpoint],
                            verbose = 1)

        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time Taken: {:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds))

        model.save_weights(filepath = "trained_models/resnet152_V2_"+str(learning_rate)+".h5")
        pickle.dump( history.history, open("trained_models/resnet152_V2_history_"+str(learning_rate)+".p", "wb" ) )

        details["model_path"] = "trained_models/resnet152_V2_"+str(learning_rate)+".h5"
        details["history_path"] = "trained_models/resnet152_V2_history_"+str(learning_rate)+".p"
        details["time_taken"] = end-start
        details["time_taken_hrs"] = "{:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds)

        resnet_models.append(details)
        
        df = pd.DataFrame(resnet_models)
        df.to_csv("resnet_results.csv", index = False)
        
    except:
        pass
    
clear_output()

## DenseNet201

In [ ]:
train_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Train',
    batch_size = 2,
    target_size = (224, 224),
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory = 'dataset/Bioconsult/32_32/Test',
    batch_size = 2,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle = False
)

for i, learning_rate in enumerate([0.000000001]):
    clear_output()
    print(learning_rate)    
    try:
        details = {
            "learning_rate": learning_rate
        }

        base_model = tf.keras.applications.DenseNet201(input_shape = (224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = True

        global_average_layer = layers.GlobalAveragePooling2D()

        classes_layer = layers.Dense(2, activation = 'sigmoid')

        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            classes_layer,
        ])

        model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      optimizer = tf.keras.optimizers.RMSprop(lr = learning_rate), #0.00000001
                      metrics=['accuracy'])

        start = time.time()

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "model_checkpoints/densenet201_"+str(learning_rate)+"{epoch:02d}.h5",
            save_weights_only=True,
            save_freq='epoch'
        )

        history = model.fit(train_generator,
                            epochs = 20,
                            validation_data=validation_generator,
                            callbacks = [checkpoint],
                            verbose = 1)

        end = time.time()
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time Taken: {:0>2}:{:0>2}:{:05.3f}".format(int(hours), int(minutes), seconds))

        model.save_weights(filepath = "trained_models/densenet201_"+str(learning_rate)+".h5")
        pickle.dump(history.history, open("trained_models/densenet201_history_"+str(learning_rate)+".p", "wb" ))
    
        details["model_path"] = "trained_models/densenet201_"+str(learning_rate)+".h5"
        details["history_path"] = "trained_models/densenet201_history_"+str(learning_rate)+".p"
        details["time_taken"] = end-start
        details["time_taken_hrs"] = "{:0>2}:{:0>2}:{:05.3f}".format(int(hours),int(minutes),seconds)

        densenet_models.append(details)
        
        df = pd.DataFrame(densenet_models)
        df.to_csv("densenet_results.csv", index = False)
    except:
        pass
    
clear_output()